In [ ]:
using Plots
using LinearAlgebra
using Printf
using LaTeXStrings
using Optim
using BenchmarkTools
using Random
using ForwardDiff
using Dierckx

In [ ]:
default(lw=2,markersize = 6,
    xtickfont=font(12), ytickfont=font(12), 
    guidefont=font(14), legendfont=font(12),titlefont=font(12))

Solve for parameters $p \in \mathbb{f}(p)$, where
$$
-u'' = f, \quad u(0) = u(1) =0,
$$
and we have noisy measurements
$$
y_i = u(\hat{x}_i) + \eta_i,
$$
at some discrete set of points $\hat{x}_i$.

We treat this as an optimziation problem of the function,
$$
J(p) = \frac{1}{2N}\sum_{i=1}^N |u(\hat{x}_i; p)-y_i|^2 + \frac{\lambda }{2} \|p\|^2,
$$
where $\lambda>0$ is a small *regularization* parameter.

In [ ]:
include("bvps1d.jl"); # include source file with matrix discretizations

In [ ]:
n = 100; # set
x = LinRange(0,1,n+1)[2:end-1];
Δx = x[2] - x[1];
x_mid = LinRange(0,1,n+1)[2:end] .- Δx;
A = assemble_spmatrix(n, Δx); # matrix discretization of d^2/dx^2


In [ ]:
function solve_ode(f)
    return A\f;
end

function evaluate_u(x, u, x_new)
    Spline1D([0;x;1], [0; u; 0],k=3)(x_new);
end

In [ ]:
p_true = Float64[3, 2];

f(p) = -p[1] * cos.(π * p[2] * x);
# f(p) = @. p[1] * exp(p[2]*x);
# f(p) = @. p[1] * sqrt(p[2]*x);

f_true = f(p_true);


u_true = solve_ode(f_true);
plot(x, u_true)

Random.seed!(10);
x_data = sort(rand(100));
# x_data = [0.3, 0.45, 0.8];
u_data = Spline1D([0; x; 1], [0; u_true; 0],k=3)(x_data)

Random.seed!(100)
γ = 1e-4;
y_data = @. u_data + γ * randn();
plot(x, u_true, label=L"True $u$")
scatter!(x_data, y_data, label="Data")
xlabel!(L"x")

In [ ]:
function F(p; λ = 1e-8)
    u = solve_ode(f(p));
    u_predict = evaluate_u(x, u, x_data);
    res = .5 * mean(abs2, u_predict-y_data) + .5 * λ * p[1]^2;
    return res
end

In [ ]:
p0= [1., 1.];
# opt = optimize(F, p0)
opt = optimize(F, p0, LBFGS())


In [ ]:
@show p_true;
@show opt.minimizer;

In [ ]:
plot(x, f_true, label=L"True $f$")
plot!(x, f(opt.minimizer),label=L"Recovered $f$")
xlabel!(L"x")

In [ ]:
u_pred = A\f(opt.minimizer);
plot(x, u_true, label=L"True $u$")
plot!(x, u_pred,label=L"Recovered $u$")
xlabel!(L"x")